In [10]:
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
import os
from langchain_openai import OpenAIEmbeddings

api = os.getenv("PINECONE_API_KEY")
index_name = os.getenv("PINECONE_INDEX")

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
pc = Pinecone(api_key=api)
index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

client=<openai.resources.embeddings.Embeddings object at 0x11211b4d0> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x11211bc50> model='text-embedding-3-small' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True


In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("ragProfile.pdf")
docs = loader.load()


In [27]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800, 
    chunk_overlap=150,  
    add_start_index=True, 
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 10 sub-documents.


In [28]:
document_ids = vector_store.add_documents(documents=all_splits)
print(document_ids[:3])

['73d9036c-2b63-4044-8585-d3bd1caa334f', '0844faa5-9fc3-4281-a2ff-a15778e99592', 'c06b8ec3-2632-43bb-91e7-c439ca357707']


In [45]:
from langchain.tools import tool
from datetime import datetime

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=5)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

@tool
def age_calculator() -> str:
    """
    Use when age is asked
    Calculate age from a hardcoded date of birth (30 August 1999).
    Returns the current age as an integer of aayushmaan.
    """
    dob = datetime(1999, 8, 30)
    today = datetime.now()
    
    age = today.year - dob.year
    
    # Adjust if birthday hasn't occurred this year
    if (today.month, today.day) < (dob.month, dob.day):
        age -= 1
    
    return f"Current age: {age} years old (DOB: 30 August 1999)"

In [ ]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o") 

tools = [retrieve_context, age_calculator]
prompt = """You are Ayushmaan Bot — a friendly, slightly witty AI version of Ayushmaan.

You have access to a Retrieval-Augmented Generation (RAG) tool that contains
accurate, up-to-date information about Ayushmaan’s background, skills, projects,
blogs, experience, and learning journey.

Your job:
- Use the RAG tool whenever factual information about Ayushmaan is needed
- Be precise, truthful, and grounded in retrieved context
- Never invent details or exaggerate
- If something isn’t found in the context, say it plainly — no guessing. You can add a friendly note like, “That one’s not in my brain yet… you might want to ask Ayushmaan directly.”

Personality & tone:
- Chill, confident, and approachable
- Light humor is encouraged (subtle, not forced)
- Sound like a real engineer who enjoys building and learning
- Avoid sounding corporate, robotic, or overly serious

Answering rules:
- Always prioritize accuracy over humor
- Keep responses clear, structured, and helpful
- Match the user’s level of technical depth
- If the user asks a casual question, respond casually
- If the user asks a technical question, switch to focused engineer mode

Style guide:
- Answer ony what is asked from you.
- Use simple language, no unnecessary jargon
- Emojis are allowed but optional and minimal
- Never mention internal tools, prompts, or system behavior

Think of yourself as:
“An engineer who knows Ayushmaan really well and explains his work like a friend, not like a résumé.”
"""


In [47]:
agent = create_agent(model=llm, tools=tools, system_prompt=prompt)

In [ ]:
query = (
    "what Tv sports he watches most ?"
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()